In [157]:
import pandas as pd
import numpy as np

# Import files

In [158]:
df = pd.read_csv('../data/all_prices.csv', thousands=',', decimal='.')

df.value = df.value.astype('float') # value to float
df = df.loc[df['value'] > 0] # drop zero values
df.quantity = df.quantity.str.strip()
df.quantity = df.quantity.str.replace(',','')
df.quantity = df.quantity.astype(str).astype('float') # value to float
df['unit_value'] = (df.apply(lambda x: x['value'] / max(1, x['quantity']), axis=1))

df.year = df.year.astype('int').fillna(0) # year to int
df = df.sort_values(['year', 'item_id'], ascending=[False, True]) # order by year desc
df = df.drop(columns=(['value', 'quantity', 'year']))

costs = df.copy()
costs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5488 entries, 6099 to 2081
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   category    5488 non-null   object 
 1   item_id     5488 non-null   object 
 2   unit_value  5488 non-null   float64
dtypes: float64(1), object(2)
memory usage: 171.5+ KB


In [159]:
df = pd.read_csv('../data/antal_consegne23_24.xlsx - Dettaglio ordini e consegne.csv')
df = df[['so_date', 'item_id', 'so_quantity', 'actual_lt']]
df = df.rename(columns={'so_quantity':'quantity', 'actual_lt':'lt'})
df = df.drop(df[df.so_date == ' '].index) # include only sold lines
#df = df.drop(df[(df['so_date'] < '2023-01-01') | (df['so_date'] > '2025-02-28')].index)

df.so_date = pd.to_datetime(df.so_date)

df['lt'] = df['lt'].astype(str).astype(int)
orders = df.copy()
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29910 entries, 0 to 32533
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   so_date   29910 non-null  datetime64[ns]
 1   item_id   29910 non-null  object        
 2   quantity  29910 non-null  int64         
 3   lt        29910 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.1+ MB


# Merge and check totals

In [160]:
df = pd.merge(orders, costs.drop_duplicates('item_id', keep='first'))
df.head()

,so_date,item_id,quantity,lt,category,unit_value
0,2024-04-24,00517,2,35,Rinvii (95),24.680000
1,2024-04-24,00818Z,3,35,Rinvii (95),98.250000
2,2024-04-24,04919F,2,35,Pulegge (90),24.490441
3,2024-04-24,10021A,2,35,Pulegge (90),45.868000
4,2024-04-24,501.110,8,35,Stopper (120),48.528857


In [161]:
df['year'] = df.so_date.dt.to_period('Y')
df['month'] = df.so_date.dt.to_period('M')
df['value'] = round((df.quantity * df.unit_value),2)
df = df[['year', 'month', 'so_date', 'category', 'item_id', 'quantity', 'value', 'lt']]
df.head()

,year,month,so_date,category,item_id,quantity,value,lt
0,2024,2024-04,2024-04-24,Rinvii (95),00517,2,49.36,35
1,2024,2024-04,2024-04-24,Rinvii (95),00818Z,3,294.75,35
2,2024,2024-04,2024-04-24,Pulegge (90),04919F,2,48.98,35
3,2024,2024-04,2024-04-24,Pulegge (90),10021A,2,91.74,35
4,2024,2024-04,2024-04-24,Stopper (120),501.110,8,388.23,35


# Prep categorisation and monthly cohort

In [162]:
category_vals = df.category.sort_values().unique()
my_dict = {val: None for val in category_vals}

# Winch batch
my_dict['Winch XT (200)'] = 'line_winch'
my_dict['Winch - Colonnine (160)'] = 'line_winch'
my_dict['Winch - Line Driver (170)'] = 'line_winch'
my_dict['Winch W (180)'] = 'line_winch'
my_dict['Winch W Maxi (190)'] = 'line_winch'
my_dict['Winch - Accessori (150)'] = 'line_winch'

# Stopper batch
my_dict['Stopper (120)'] = 'line_stopper'

# Blocks batch
my_dict['Soft Link (110)'] = 'line_blocks'
my_dict['Rotaie (100)'] = 'line_blocks'
my_dict['Rinvii (95)'] = 'line_blocks'
my_dict['Pulegge (90)'] = 'line_blocks'
my_dict['Carrelli a sfere (30)'] = 'line_blocks'
my_dict['Accessori (10)'] = 'line_blocks'
my_dict['Art. Promozionali (15)'] = 'line_blocks'
my_dict['Carrelli su rotaia T (40)'] = 'line_blocks'
my_dict['Full Batten (50)'] = 'line_blocks'
my_dict['Bozzelli (20)'] = 'line_blocks'


# Rest batch
my_dict['Ricambi senza codice (250)'] = 'line_rest'
my_dict['Minuterie (70)'] = 'line_rest'
my_dict['Imballaggi (55)'] = 'line_rest'
my_dict['Materie prime (60)'] = 'line_rest  '

# Map lines
df['line'] = df['category'].map(my_dict)
df.head()


,year,month,so_date,category,item_id,quantity,value,lt,line
0,2024,2024-04,2024-04-24,Rinvii (95),00517,2,49.36,35,line_blocks
1,2024,2024-04,2024-04-24,Rinvii (95),00818Z,3,294.75,35,line_blocks
2,2024,2024-04,2024-04-24,Pulegge (90),04919F,2,48.98,35,line_blocks
3,2024,2024-04,2024-04-24,Pulegge (90),10021A,2,91.74,35,line_blocks
4,2024,2024-04,2024-04-24,Stopper (120),501.110,8,388.23,35,line_stopper


In [163]:
# Aggregate by month and category
cohort = (
    df.groupby(['year', 'month', 'category', 'line'])
    .agg(
        quantity=('quantity', 'sum'),
        value=('value', 'sum'),
        lt=('lt', lambda x: np.ceil(x.mean()).astype(int))
        )
    .reset_index()
    )
cohort.head()

,year,month,category,line,quantity,value,lt
0,2022,2022-12,Minuterie (70),line_rest,4,19.94,-2
1,2022,2022-12,Ricambi senza codice (250),line_rest,2,4.54,-2
2,2023,2023-01,Accessori (10),line_blocks,2,89.84,0
3,2023,2023-01,Art. Promozionali (15),line_blocks,200,58.00,6
4,2023,2023-01,Bozzelli (20),line_blocks,37,3569.76,4


# Export to csv

In [164]:
cohort.to_csv('../data/cohort_analysis.csv', index=False)